In [1]:
import numpy as np
from keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

In [2]:
max_features = 20000
maxlen = 100
batch_size = 32
embedding_size = 128
lstm_size = 64
epochs = 5

In [3]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

17464789/17464789 [==============================] - 4s 0us/step


In [4]:
X_train = pad_sequences(X_train, maxlen=maxlen, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=maxlen, padding='post', truncating='post')

In [5]:
model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=maxlen))
model.add(Bidirectional(LSTM(lstm_size, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(1, activation='sigmoid'))

In [6]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/5
782/782 [==============================] - 175s 218ms/step - loss: 0.4803 - accuracy: 0.7607 - val_loss: 0.4218 - val_accuracy: 0.8076
Epoch 2/5
782/782 [==============================] - 169s 216ms/step - loss: 0.2821 - accuracy: 0.8874 - val_loss: 0.4442 - val_accuracy: 0.8003
Epoch 3/5
782/782 [==============================] - 172s 220ms/step - loss: 0.1863 - accuracy: 0.9300 - val_loss: 0.4901 - val_accuracy: 0.7977
Epoch 4/5
782/782 [==============================] - 181s 232ms/step - loss: 0.1184 - accuracy: 0.9585 - val_loss: 0.6535 - val_accuracy: 0.7991
Epoch 5/5
782/782 [==============================] - 204s 261ms/step - loss: 0.0744 - accuracy: 0.9738 - val_loss: 0.7692 - val_accuracy: 0.7843


In [8]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy: %.2f%%" % (scores[1]*100))

Test Accuracy: 78.43%


In [9]:
import re
def clean_text(text):
    text = re.sub('[^a-zA-Z]+', ' ', text)
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    return text

In [10]:
def predict_sentiment(model, tokenizer, text):
    text = clean_text(text)
    text = [text]
    text = tokenizer.texts_to_sequences(text)
    text = pad_sequences(text, maxlen=maxlen, padding='post', truncating='post')
    
    sentiment = model.predict(text)
    sentiment = np.argmax(sentiment, axis=-1)
    
    return 'Positive' if sentiment == 1 else 'Negative'

In [11]:
from keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer()

In [14]:
text = "I hate the movie. It was not great!"
sentiment = predict_sentiment(model, tokenizer, text)
print(sentiment) # output: Negative

1/1 [==============================] - 0s 26ms/step
Negative


In [13]:
text = "It was bad experience."
sentiment = predict_sentiment(model, tokenizer, text)
print(sentiment) # output: Negative

1/1 [==============================] - 0s 29ms/step
Negative
